In [ ]:
import os
import json
import cv2
import numpy as np
import shutil
from pathlib import Path
from tqdm import tqdm

class OBBConverter:
    def __init__(self, input_dir, output_dir):
        self.input_dir = input_dir
        self.output_dir = output_dir
        self.class_map = {
            "person": 0,
            "gun": 1,
            "knife": 2,
            "stick": 3
        }
        self.image_extensions = [".jpg", ".jpeg", ".png", ".bmp"]
        
    def setup_directories(self):
        os.makedirs(self.output_dir, exist_ok=True)
    
    def get_file_pairs(self):
        file_pairs = []
        for file in os.listdir(self.input_dir):
            file_path = os.path.join(self.input_dir, file)
            if not os.path.isfile(file_path):
                continue
                
            base_name, ext = os.path.splitext(file)
            if ext.lower() not in self.image_extensions:
                continue
                
            json_path = os.path.join(self.input_dir, f"{base_name}.json")
            if os.path.exists(json_path):
                file_pairs.append((file_path, json_path))
        
        return file_pairs
    
    def fit_obb_to_points(self, points, img_width, img_height):
        if len(points) < 3:
            if len(points) == 2:
                p1, p2 = points
                p3 = [p1[0], p2[1]]
                p4 = [p2[0], p1[1]]
                points = [p1, p3, p2, p4]
            else:
                return None
        
        points_array = np.array(points, dtype=np.float32)
        
        if len(points) > 4:
            distances = []
            center = np.mean(points_array, axis=0)
            for point in points_array:
                dist = np.linalg.norm(point - center)
                distances.append(dist)
            
            avg_dist = np.mean(distances)
            std_dist = np.std(distances)
            
            filtered_points = []
            for i, point in enumerate(points_array):
                if distances[i] <= avg_dist + 2 * std_dist:
                    filtered_points.append(point)
            
            if len(filtered_points) >= 3:
                points_array = np.array(filtered_points, dtype=np.float32)
        
        rect = cv2.minAreaRect(points_array)
        box = cv2.boxPoints(rect)
        
        result = []
        for point in box:
            x = max(0, min(float(point[0]), img_width - 1))
            y = max(0, min(float(point[1]), img_height - 1))
            result.append([x, y])
        
        return result
    
    def convert_json_to_obb(self, json_path, output_path):
        try:
            with open(json_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            img_width = data.get("imageWidth", 0)
            img_height = data.get("imageHeight", 0)
            
            if img_width <= 0 or img_height <= 0:
                return False
            
            output_shapes = []
            
            for shape in data.get("shapes", []):
                label = shape.get("label", "")
                shape_type = shape.get("shape_type", "")
                points = shape.get("points", [])
                
                if shape_type == "rectangle" and len(points) == 2:
                    x1, y1 = points[0]
                    x2, y2 = points[1]
                    box_points = [[x1, y1], [x2, y1], [x2, y2], [x1, y2]]
                    obb_points = self.fit_obb_to_points(box_points, img_width, img_height)
                    
                elif shape_type == "polygon":
                    obb_points = self.fit_obb_to_points(points, img_width, img_height)
                    
                else:
                    continue
                
                if obb_points is not None:
                    output_shape = {
                        "label": label,
                        "shape_type": "polygon",
                        "points": obb_points,
                        "group_id": None,
                        "flags": {}
                    }
                    output_shapes.append(output_shape)
            
            output_data = data.copy()
            output_data["shapes"] = output_shapes
            
            with open(output_path, 'w', encoding='utf-8') as f:
                json.dump(output_data, f, indent=2)
            
            return True
        except Exception as e:
            print(f"Error processing {json_path}: {e}")
            return False
    
    def process_dataset(self):
        self.setup_directories()
        file_pairs = self.get_file_pairs()
        
        for img_path, json_path in tqdm(file_pairs, desc="Processing"):
            img_filename = os.path.basename(img_path)
            json_filename = os.path.basename(json_path)
            
            output_json = os.path.join(self.output_dir, json_filename)
            output_img = os.path.join(self.output_dir, img_filename)
            
            if self.convert_json_to_obb(json_path, output_json):
                shutil.copy(img_path, output_img)
        
        processed_count = len(file_pairs)
        print(f"Processed {processed_count} files")
        print(f"Results saved to {self.output_dir}")

def main():
    input_dir = r"D:\bai_toan\weapon\13_4\tong\DJI_0382_W\DJI_0382_W\d\new\output"
    output_dir = r"D:\bai_toan\weapon\13_4\tong\DJI_0382_W\DJI_0382_W\d\new\problem"
    converter = OBBConverter(input_dir, output_dir)
    converter.process_dataset()

if __name__ == "__main__":
    main()

Processing: 100%|██████████| 3481/3481 [05:17<00:00, 10.96it/s]

Processed 3481 files
Results saved to D:\bai_toan\weapon\13_4\tong\DJI_0382_W\DJI_0382_W\d\new\problem


# obb json -> obb txt

In [5]:
import os
import json
import numpy as np
from pathlib import Path
from tqdm import tqdm
import math
import shutil

class LabelmeToYOLOOBBConverter:
    def __init__(self, input_dir, output_dir):
        self.input_dir = input_dir
        self.output_dir = output_dir
        self.class_map = {
            "person": 0,
            "gun": 1,
            "knife": 2,
            "stick": 3
        }
        self.image_extensions = [".jpg", ".jpeg", ".png", ".bmp"]
        
    def setup_directories(self):
        os.makedirs(self.output_dir, exist_ok=True)
    
    def get_json_files(self):
        json_files = []
        for file in os.listdir(self.input_dir):
            if file.endswith(".json"):
                json_path = os.path.join(self.input_dir, file)
                json_files.append(json_path)
        return json_files
    
    def order_points_clockwise(self, points):
        center = np.mean(points, axis=0)
        angles = []
        for point in points:
            angle = math.atan2(point[1] - center[1], point[0] - center[0])
            angles.append(angle)
        
        sorted_indices = np.argsort(angles)
        sorted_points = [points[i] for i in sorted_indices]
        
        return sorted_points
    
    def normalize_points(self, points, img_width, img_height):
        normalized_points = []
        for point in points:
            x, y = point
            norm_x = x / img_width
            norm_y = y / img_height
            normalized_points.append([norm_x, norm_y])
        return normalized_points
    
    def convert_json_to_yolo_obb(self, json_path):
        try:
            with open(json_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            img_width = data.get("imageWidth", 0)
            img_height = data.get("imageHeight", 0)
            
            if img_width <= 0 or img_height <= 0:
                return False, "Invalid image dimensions"
            
            base_name = os.path.splitext(os.path.basename(json_path))[0]
            output_txt_path = os.path.join(self.output_dir, f"{base_name}.txt")
            
            yolo_lines = []
            
            for shape in data.get("shapes", []):
                label = shape.get("label", "")
                shape_type = shape.get("shape_type", "")
                points = shape.get("points", [])
                
                if shape_type != "polygon" or label not in self.class_map:
                    continue
                
                class_idx = self.class_map[label]
                
                if len(points) != 4:
                    continue
                
                ordered_points = self.order_points_clockwise(points)
                norm_points = self.normalize_points(ordered_points, img_width, img_height)
                
                line = f"{class_idx}"
                for point in norm_points:
                    line += f" {point[0]:.6f} {point[1]:.6f}"
                yolo_lines.append(line)
            
            with open(output_txt_path, 'w', encoding='utf-8') as f:
                for line in yolo_lines:
                    f.write(line + "\n")
            
            return True, None
            
        except Exception as e:
            return False, str(e)
    
    def process_dataset(self):
        self.setup_directories()
        json_files = self.get_json_files()
        
        success_count = 0
        error_count = 0
        
        for json_path in tqdm(json_files, desc="Converting to YOLOv8 OBB format"):
            success, error = self.convert_json_to_yolo_obb(json_path)
            
            if success:
                success_count += 1
                
                base_name = os.path.splitext(os.path.basename(json_path))[0]
                for ext in self.image_extensions:
                    img_path = os.path.join(self.input_dir, f"{base_name}{ext}")
                    if os.path.exists(img_path):
                        img_output_path = os.path.join(self.output_dir, f"{base_name}{ext}")
                        try:
                            shutil.copy(img_path, img_output_path)
                        except Exception as e:
                            print(f"Warning: Failed to copy image {img_path}: {e}")
                        break
            else:
                error_count += 1
                print(f"Error processing {json_path}: {error}")
        
        print(f"Conversion complete: {success_count} files converted successfully, {error_count} errors")
        print(f"Results saved to {self.output_dir}")

def main():
    input_dir = r"D:\bai_toan\weapon\3_26\tong\tong_seg\new\problem"
    output_dir = r"D:\bai_toan\weapon\3_26\tong\tong_seg\new\output"
    converter = LabelmeToYOLOOBBConverter(input_dir, output_dir)
    converter.process_dataset()

if __name__ == "__main__":
    main()

Converting to YOLOv8 OBB format: 100%|██████████| 46964/46964 [31:10<00:00, 25.10it/s]  

Conversion complete: 46964 files converted successfully, 0 errors
Results saved to D:\bai_toan\weapon\3_26\tong\tong_seg\new\output
